# Utilization Prediction: Prediction of hourly utilization of the two sites

## Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
os.environ["KERAS_BACKEND"] = "torch"

import keras
from keras import Sequential
from keras.layers import Dense, Dropout

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

## Load and prepare data

In [2]:
# Load data
charging_data = pd.read_pickle("data/charging_modified.pkl")

# Drop not important columns in charging data
charging_data.drop(columns = ["NoChargingTime", "NoChargingTimeMinutes", "userRegistered", "spaceID", "stationID", "ChargingTime", 
                              "user_paymentRequired_values", "user_requestedDeparture", "userID", "ChargingTimeMinutes", "user_kWhRequestFulfilment", 
                              "fullyCharged", "user_kWhRequested", "user_milesRequested", "user_minutesAvailable", "month", "weekday", 
                              "NoChargingTimeHours", "ChargingTimeHours", "disconnectTime"], inplace = True)

# Delete rows with null values
charging_data = charging_data.dropna()

charging_data = charging_data.astype({
    "siteID": "int64"
})

charging_data  = charging_data[:31670]

# Delete duplicate row index
charging_data.reset_index(drop = True, inplace = True)
charging_data.loc[:, "id"] = charging_data.index
charging_data

,connectionTime,doneChargingTime,kWhDelivered,siteID,kWhPerMinute,user_WhPerMile,id
0,2018-04-30 15:07:49+00:00,2018-05-01 00:27:51+00:00,47.808,2,0.085366,350.0,0
1,2018-05-07 14:38:18+00:00,2018-05-08 01:14:53+00:00,27.683,2,0.043487,400.0,1
2,2018-05-11 15:17:01+00:00,2018-05-11 23:05:56+00:00,17.485,2,0.037288,350.0,2
3,2018-05-14 13:50:26+00:00,2018-05-14 20:43:40+00:00,11.795,2,0.028543,400.0,3
4,2018-05-15 00:06:39+00:00,2018-05-15 00:39:01+00:00,3.076,2,0.095036,250.0,4
...,...,...,...,...,...,...,...
31665,2019-12-20 13:52:05+00:00,2019-12-20 17:42:07+00:00,11.190,1,0.048645,600.0,31665
31666,2019-12-20 14:24:31+00:00,2019-12-20 16:44:51+00:00,13.371,1,0.095280,400.0,31666
31667,2019-12-20 15:15:33+00:00,2019-12-20 22:53:22+00:00,36.269,1,0.079222,333.0,31667
31668,2019-12-20 15:43:42+00:00,2019-12-20 17:29:05+00:00,5.054,1,0.047958,333.0,31668


## Create feature vector X and labels Y

In [3]:
# Split time interval between connectionTime and doneChargingTime in minutes for every hour
def split_time_interval(index, start_date, end_date):
    indices = pd.DatetimeIndex([start_date])
    indices = indices.append(pd.date_range(start_date.ceil(freq = "H"), end_date.floor(freq = "H"), freq = "H"))
    indices = indices.append(pd.DatetimeIndex([end_date]))

    time_list = list(map(lambda x : 60 if x == 0 else x, indices.minute))
    time_list[0] = 60 - time_list[0]
    
    return list((time_list, indices))


# Create table with session id, charging time in minutes per hour, timestamps and siteID
def create_minute_table(id, minute_array, siteID):
    return list(zip([id] * len(minute_array[0]), minute_array[0], minute_array[1], [siteID] * len(minute_array[0])))

# Group entries by timestamp
def group_by_timestamp(X):
    X = X.groupby("timestamp").agg({ "siteID": "mean", "kWhDemandPerHour": "sum"})
    return X

In [ ]:
# Concat single minute lists to one big list with lists of tuples containing id, charging times in minutes per, timestamps and siteID
temp = []

for index in charging_data.index :
    charging_times = split_time_interval(charging_data.index[index], 
                                         charging_data.loc[index, "connectionTime"], 
                                         charging_data.loc[index, "doneChargingTime"])
    temp.append(create_minute_table(charging_data.loc[index, "id"], charging_times, charging_data.loc[index, "siteID"]))
    
charging_data.drop("siteID", inplace = True, axis = 1)

# Parse list of lists into four lists: ids, charging time in minutes per hour, timestamps and siteIDs
ids, charging_times, timestamps, siteIDs = map(list, zip(*[tuple for list in temp for tuple in list]))

# Create data.frame with four columns: session id, charging time, starting timestamp in minutes for every hour and siteID
charging_time_per_hour = pd.DataFrame(
                            {"id": ids,
                             "charging_time_per_hour": charging_times,
                             "timestamp": timestamps,
                             "siteID": siteIDs})

charging_time_per_hour.loc[:, "timestamp"] = charging_time_per_hour.loc[:, "timestamp"].dt.floor(freq = "H")

### Labels Y:

In [ ]:
# Join charge time per hour with the rest of the charging data
temp = charging_time_per_hour.merge(charging_data, how = "left", on = "id").drop(["connectionTime", "id"], axis = 1)

# Create column for delivered kWh per hour interval
temp["kWhDemandPerHour"] = temp["charging_time_per_hour"] * temp["kWhPerMinute"]

# Divide data set into two sets: one for each site
temp1 = temp.loc[temp["siteID"] == 1]
temp2 = temp.loc[temp["siteID"] == 2]

# Group entries by timestamp
temp1 = group_by_timestamp(temp1)
temp2 = group_by_timestamp(temp2)

# Combine the sets together into one
temp = pd.concat([temp1, temp2], axis = 0)

temp["timestamp"] = temp.index
temp.reset_index(drop = True, inplace = True)

In [ ]:
# Define labels Y
Y = temp[["kWhDemandPerHour", "timestamp", "siteID"]]
Y

### Feature Vector X:

In [ ]:
def encode_to_cyclic(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

def expand_datetime_to_columns(data, col):
    data[col + 'Month'] = data[col].dt.month
    data[col + 'Day'] = data[col].dt.day
    data[col + 'Hour'] = data[col].dt.hour
    return data

X = temp.drop(["kWhDemandPerHour"],  axis = 1)

# Split connection into year, month, day, hour
X = expand_datetime_to_columns(X, 'timestamp')

# Encode the time components to cyclic
X = encode_to_cyclic(X, 'timestampMonth', 12)

X = encode_to_cyclic(X, 'timestampDay', 31)

X = encode_to_cyclic(X, 'timestampHour', 24)

X.drop(['timestampMonth', 'timestampDay', 'timestampHour'], axis = 1, inplace = True)

# Add weekday, month and hour as categorical variable
#X["weekday"] = X["timestamp"].dt.day_name()
#X["month"] = X["timestamp"].dt.month_name()
#X["hour"] = X["timestamp"].dt.hour

X = X.fillna(0)

# Create dummies for categorical variables
X = X.astype({
    "siteID": "int64"
})

X = pd.get_dummies(X, columns = ["siteID"])
#X = pd.get_dummies(X, columns = ["siteID", "weekday", "month", "hour"])

# Sort values after siteID and timestamp in ascending order
X.sort_values(["siteID_2", "timestamp"])
X

## Cross-validation: split data in training and test set

In [ ]:
# Split data into training set and testing set and normalize
scaler = StandardScaler()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.3, random_state = 30)

X_train_scaled = scaler.fit_transform(X_train.drop("timestamp", axis = 1))
X_test_scaled = scaler.transform(X_test.drop("timestamp", axis = 1))

# Create X sets for testing performance for each site
X_test_site1 = X_test[X_test["siteID_1"] == True].sort_values(["timestamp"])
X_test_site2 = X_test[X_test["siteID_2"] == True].sort_values(["timestamp"])

# Create Y sets for testing performance for each site
Y_test_site1 = Y_test[Y_test["siteID"] == 1.0].sort_values("timestamp").drop("timestamp", axis = 1).loc[:, "kWhDemandPerHour"]
Y_test_site2 = Y_test[Y_test["siteID"] == 2.0].sort_values("timestamp").drop("timestamp", axis = 1).loc[:, "kWhDemandPerHour"]

# Extract kWhDemandPerHour as labels
Y_train = Y_train.loc[:, "kWhDemandPerHour"]
Y_test = Y_test.loc[:, "kWhDemandPerHour"]

## Prediction model 1: Neural Network

In [ ]:
# Build the neural network
network = Sequential()

# Add layers
network.add(Dense(input_shape = (45,), units = 21, activation = "relu"))
network.add(Dense(units = 21, activation = "relu"))
network.add(Dense(units = 1, activation = "relu"))

In [ ]:
# Compile the neural network
network.compile(optimizer = "adam", 
                   loss = "mean_squared_error", 
                   metrics = ["mean_squared_error"])
network.summary()

In [ ]:
# Fit the Neural Network
network.fit(X_train_scaled, Y_train, batch_size=50, epochs=200)

### Performance evaluation

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score

# Evaluate performance
mse_nn  = mean_squared_error(Y_test, network.predict(X_test_scaled))
mae_nn  = mean_absolute_error(Y_test, network.predict(X_test_scaled))
msle_nn = mean_squared_log_error(Y_test, network.predict(X_test_scaled))
print("Mean squared error: " + str(mse_nn))
print("Mean absolute error: " + str(mae_nn))
print("Mean squared log error: " + str(msle_nn))

### Performance Visualization

In [ ]:
predictions_site1 = network.predict(scaler.transform(X_test_site1.drop("timestamp", axis = 1)))[:, 0]
timestamps = X_test_site1["timestamp"].sort_values()

# Visualization of the predicted value and the observed value over time for site 1
plt.figure(figsize = (25,6))

plt.plot(timestamps, predictions_site1, c = "blue", label = "predicted values")
plt.plot(timestamps, Y_test_site1, c = "red", label = "observed values")

plt.xlabel("Time")
plt.ylabel("kWh demand")
plt.title("Predicted and observed kWh demand per hour for site 1 (Neural Network)")
plt.legend()

plt.show()

In [ ]:
# Visualization of the predicted value and the observed value over 48 hours for site 1 (to visualize it better)
plt.figure(figsize = (15,6))

plt.plot(timestamps.iloc[0:48], predictions_site1[0:48], c = "blue", label = "predicted values")
plt.plot(timestamps.iloc[0:48], Y_test_site1[0:48], c = "red", label = "observed values")

plt.xlabel("Time")
plt.ylabel("kWh demand")
plt.title("Predicted and observed kWh demand per hour for site 1 over 2 days (Neural Network)")
plt.legend()

plt.show()

In [ ]:
predictions_site2 = network.predict(scaler.transform(X_test_site2.drop("timestamp", axis = 1)))[:, 0]
timestamps = X_test_site2["timestamp"].sort_values()

# Visualization of the predicted value and the observed value over time for site 2
plt.figure(figsize = (25,6))

plt.plot(timestamps, predictions_site2, c = "blue", label = "predicted values")
plt.plot(timestamps, Y_test_site2, c = "red", label = "observed values")

plt.xlabel("Time")
plt.ylabel("kWh demand")
plt.title("Predicted and observed kWh demand per hour for site 2 (Neural Network)")
plt.legend()

plt.show()

In [ ]:
# Visualization of the predicted value and the observed value over 48 hours for site 2 (to visualize it better)
plt.figure(figsize = (15,6))

plt.plot(timestamps.iloc[0:48], predictions_site2[0:48], c = "blue", label = "predicted values")
plt.plot(timestamps.iloc[0:48], Y_test_site2[0:48], c = "red", label = "observed values")

plt.xlabel("Time")
plt.ylabel("kWh demand")
plt.title("Predicted and observed kWh demand per hour for site 2 over 2 days (Neural Network)")
plt.legend()

plt.show()

## Prediction Model 2: Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import graphviz

# fit decision tree for regression
reg_tree = DecisionTreeRegressor(max_depth = 15, criterion = "squared_error")
reg_tree.fit(X_train_scaled, Y_train)

In [ ]:
# Plot tree
data = export_graphviz(reg_tree, feature_names = X.drop("timestamp", axis = 1).columns)
graph = graphviz.Source(data)

graph

### Performance Evaluation

In [ ]:
# Evaluate performance
mse_dt    = mean_squared_error(Y_test, reg_tree.predict(X_test_scaled))
mae_dt    = mean_absolute_error(Y_test, reg_tree.predict(X_test_scaled))
r2_dt     = r2_score(Y_test, reg_tree.predict(X_test_scaled))
print("Mean squared error: " + str(mse_dt))
print("Mean absolute rror: " + str(mae_dt))
print("R2 score: " + str(r2_dt))

print(np.mean(Y_test))

### Performance Visualization

In [ ]:
predictions_site1_tree = reg_tree.predict(scaler.transform(X_test_site1.drop("timestamp", axis = 1)))
timestamps = X_test_site1["timestamp"].sort_values()

# Visualization of difference between predicted and observed utilization rates for site 1
plt.figure(figsize = (25,6))

plt.plot(timestamps, predictions_site1_tree, c = "blue", label = "predicted values")
plt.plot(timestamps, Y_test_site1, c = "red", label = "observed values")

plt.xlabel("Time")
plt.ylabel("kWh demand")
plt.title("Predicted and observed kWh demand per hour for site 1 (Decision Tree Regressor)")
plt.legend()

plt.show()

In [ ]:
# Visualization of the predicted value and the observed value over 48 hours for site 1 (to visualize it better)
plt.figure(figsize = (15,6))

plt.plot(timestamps[0:48], predictions_site1_tree[0:48], c = "blue", label = "predicted values")
plt.plot(timestamps[0:48], Y_test_site1[0:48], c = "red", label = "observed values")

plt.xlabel("Time")
plt.ylabel("kWh demand")
plt.title("Predicted and observed kWh demand per hour for site 1 over 2 days (Decision Tree Regressor)")
plt.legend()

plt.show()

In [ ]:
predictions_site2_tree = reg_tree.predict(scaler.transform(X_test_site2.drop("timestamp", axis = 1)))
timestamps = X_test_site2["timestamp"].sort_values()

# Visualization of difference between predicted and observed utilization rates for site 2
plt.figure(figsize = (25,6))

plt.plot(timestamps, predictions_site2_tree, c = "blue", label = "predicted values")
plt.plot(timestamps, Y_test_site2, c = "red", label = "observed values")

plt.xlabel("Time")
plt.ylabel("kWh demand")
plt.title("Predicted and observed kWh demand per hour for site 2 (Decision Tree Regressor)")
plt.legend()

plt.show()

In [ ]:
# Visualization of the predicted value and the observed value over 48 hours for site 2 (to visualize it better)
plt.figure(figsize = (15,6))

plt.plot(timestamps[0:48], predictions_site2_tree[0:48], c = "blue", label = "predicted values")
plt.plot(timestamps[0:48], Y_test_site2[0:48], c = "red", label = "observed values")

plt.xlabel("Time")
plt.ylabel("kWh demand")
plt.title("Predicted and observed kWh demand per hour for site 2 over 2 days (Decision Tree Regressor)")
plt.legend()

plt.show()

## Business Case Example

### Site 1 prediction for march 2024

In [ ]:
# generate data for the first two weeks of march 2024 for site 1
date_range = pd.DataFrame({"timestamp": pd.date_range("2024-03-01", periods = 336, freq = "h")})

data = pd.DataFrame()
data["siteID_1"] = [True] * date_range.size
data["siteID_2"] = [False] * date_range.size

data["weekday"] = date_range["timestamp"].dt.day_name()
data = pd.get_dummies(data, columns = ["weekday"])

data["month_April"] = [False] * date_range.size
data["month_August"] = [False] * date_range.size
data["month_December"] = [False] * date_range.size
data["month_February"] = [False] * date_range.size
data["month_January"] = [False] * date_range.size
data["month_July"] = [False] * date_range.size
data["month_June"] = [False] * date_range.size
data["month_March"] = [True] * date_range.size
data["month_May"] = [False] * date_range.size
data["month_November"] = [False] * date_range.size
data["month_October"] = [False] * date_range.size
data["month_September"] =[False] * date_range.size
data["hour"] = date_range["timestamp"].dt.hour

data = pd.get_dummies(data, columns = ["hour"])

predictions_site1 = reg_tree.predict(scaler.transform(data))

In [ ]:
plt.figure(figsize = (25,6))
plt.plot(date_range, predictions_site1)

plt.xlabel("Time")
plt.ylabel("kWh demand per hour")
plt.title("kWh demand prediction for the first two weeks of March for site 1")

plt.show()

In [ ]:
# generate data for the first two weeks of march 2024 for site 2
data["siteID_1"] = [False] * date_range.size
data["siteID_2"] = [True] * date_range.size

predictions_site2 = reg_tree.predict(scaler.transform(data))

plt.figure(figsize = (25,6))
plt.plot(date_range, predictions_site2)

plt.xlabel("Time")
plt.ylabel("kWh demand per hour")
plt.title("kWh demand prediction for the first two weeks of March for site 2")

plt.show()